In [1]:
import os
import pandas as pd
import numpy as np

In [5]:
folder_path = 'Visual Analytics/VAST-Challenge-2022/VA - Datasets/Activity Logs'

dfs = []

for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        file_path = os.path.join(folder_path, file_name)
        
        # Loading the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Adding the DataFrame to the list
        dfs.append(df)

# Union of all DataFrames in the list into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113923735 entries, 0 to 113923734
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   timestamp          object 
 1   currentLocation    object 
 2   participantId      float64
 3   currentMode        object 
 4   hungerStatus       object 
 5   sleepStatus        object 
 6   apartmentId        float64
 7   availableBalance   float64
 8   jobId              float64
 9   financialStatus    object 
 10  dailyFoodBudget    float64
 11  weeklyExtraBudget  float64
dtypes: float64(6), object(6)
memory usage: 10.2+ GB


In [6]:
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])

merged_df['weekday'] = merged_df['timestamp'].dt.day_name()

merged_df['weekday'] = merged_df['weekday'].replace(['Monday','Tuesday','Wednesday','Thursday','Friday'],'Weekday')

merged_df['weekday'] = merged_df['weekday'].replace(['Saturday','Sunday'],'Weekend')

merged_df['weekday'].value_counts()
    

Weekday    81553526
Weekend    32370209
Name: weekday, dtype: int64

## Week and week-end days

In [18]:
check_df = pd.read_csv("Visual Analytics/VAST-Challenge-2022/VA - Datasets/CheckinJournal.csv")

In [24]:
# timestamp conversion
check_df['timestamp'] = pd.to_datetime(check_df['timestamp'])
merged_df['timestamp'] = pd.to_datetime(check_df['timestamp'])

In [ ]:
final = pd.merge(merged_df, check_df, on=['participantId', 'timestamp'], how='inner')

# Count of participantId in the currentLocation in a given timestamp
result_df = final.groupby(['timestamp', 'currentLocation']).size().reset_index(name='count')

In [46]:
# New column where 'weekend' if the day is Saturday or Sunday and 'weekday' otherwise
result_df['weekday'] = result_df['timestamp'].dt.day_name()

result_df['weekday'] = result_df['weekday'].replace(['Monday','Tuesday','Wednesday','Thursday','Friday'],'Weekday')
result_df['weekday'] = result_df['weekday'].replace(['Saturday','Sunday'],'Weekend')

In [58]:
# Count of people at a given weekday location
count_feriali = result_df[result_df['weekday'] == 'Weekday'].groupby('currentLocation')['count'].sum().reset_index()
# Count of people at a given weekend location
count_festivi = result_df[result_df['weekday'] == 'Weekend'].groupby('currentLocation')['count'].sum().reset_index()

In [66]:
count_feriali.to_csv('public/weekdays.csv', index=False)

In [67]:
count_festivi.to_csv('public/weekends.csv', index=False)

## Travel 

In [ ]:
df = pd.read_csv("Visual Analytics/VAST-Challenge-2022/VA - Datasets/TravelJournal.csv")
travel = df[['travelEndTime', 'travelStartTime']].copy()

In [ ]:
travel = travel.sort_values(by=['travelStartTime'])

In [ ]:
travel['travelEndTime'] = pd.to_datetime(travel['travelEndTime'])
travel['travelStartTime'] = pd.to_datetime(travel['travelStartTime'])
travel['travelDuration'] = travel['travelEndTime'] - travel['travelStartTime']
travel['travelDuration'] = travel['travelDuration'].dt.total_seconds().div(60).astype(int)

In [ ]:
df_new = travel.groupby(travel['travelStartTime'].dt.date)['travelDuration'].sum().reset_index()

# Rename'travelStartTime' into 'date'
df_new = df_new.rename(columns={'travelStartTime': 'date'})

In [ ]:
df_new.to_csv('public/travelDuration.csv', index=False)

## Routines

In [ ]:
def hamming_distance(seq1, seq2):
    return sum(el1 != el2 for el1, el2 in zip(seq1, seq2))

def find_similar_participants(grouped_df):
    min_distance = float('inf')
    max_distance = 0
    most_similar_participants = ()
    least_similar_participants = ()

    for i in range(len(grouped_df)):
        for j in range(i + 1, len(grouped_df)):
            seq1 = grouped_df['currentMode'].iloc[i]
            seq2 = grouped_df['currentMode'].iloc[j]

            distance = hamming_distance(seq1, seq2)
            
            if distance < min_distance:
                min_distance = distance
                most_similar_participants = (grouped_df['participantId'].iloc[i], grouped_df['participantId'].iloc[j])
                
            if distance > max_distance:
                max_distance = distance
                least_similar_participants = (grouped_df['participantId'].iloc[i], grouped_df['participantId'].iloc[j])

    return most_similar_participants, least_similar_participants


In [ ]:
most_similar, least_similar = find_similar_participants(merged_df)
print("Partecipanti più simili:", most_similar)
print("Partecipanti meno simili:", least_similar)

In [ ]:
routine = merged_df[['participantId', 'timestamp', 'currentMode']].copy()

In [11]:
routine.to_csv('public/routine.csv', index=False)

In [2]:
df_routine = pd.read_csv('public/routine.csv')

In [ ]:
df_routine_179 = df_routine[df_routine['participantId'] == 179]
df_routine_728 = df_routine[df_routine['participantId'] == 728]
df_routine_902 = df_routine[df_routine['participantId'] == 902]
df_routine_911 = df_routine[df_routine['participantId'] == 911]

In [ ]:
df_routine_179.to_csv('public/routine_179.csv', index=False)

In [ ]:
df_routine_902.to_csv('public/routine_902.csv', index=False)

In [ ]:
df_routine_728.to_csv('public/routine_728.csv', index=False)

In [ ]:
df_routine_911.to_csv('public/routine_911.csv', index=False)

## Turnover

In [ ]:
df = pd.read_csv("Visual Analytics/VAST-Challenge-2022/VA - Datasets/Jobs.csv")

In [ ]:
partecipants = merged_df[['jobId','participantId','timestamp']].copy()

In [ ]:
turnover = df[['jobId','hourlyRate']].copy()

In [ ]:
# rounds hourlyRate to 2 decimal places
turnover['hourlyRate'] = turnover['hourlyRate'].round(2)

In [ ]:
turn = pd.merge(partecipants, turnover, on='jobId')
tover = turn[['jobId','participantId','hourlyRate']].copy()

In [ ]:
# where participanId and jobId are the same, take only the first line
tover = df.drop_duplicates(subset=['participantId', 'jobId'], keep='first')

In [ ]:
# eliminates the last occurrence of each participantId
last_occurrences = tover.drop_duplicates(subset=['participantId'], keep='last').index
over = tover.drop(last_occurrences)

In [ ]:
df = over.groupby(['jobId', 'hourlyRate']).size().reset_index(name='counts')

In [ ]:
df.to_csv('public/turnover.csv', index=False)